In [4]:

import torch
#from datasets import load_dataset
from tokenizers import tokenizers
from tokenizers.models import WordLevel
from torch.utils.data import Dataset, DataLoader, random_split

from datasets import load_dataset
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.trainers import WordLevelTrainer
from tokenizers.pre_tokenizers import Whitespace

import random
random.seed(42)

ModuleNotFoundError: No module named 'datasets'

In [5]:
!pip install datasets


INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.9 MB/s eta 0:00:00


In [7]:

from datasets import load_dataset
def get_config():
    return {
        "batch_size": 8,
        "num_epochs": 20,
        "lr": 10**-4,
        "seq_len": 350,
        "d_model": 512,
        "datasource": 'opus_books',
        "lang_src": "en",
        "lang_tgt": "it",
        "model_folder": "weights",
        "model_basename": "tmodel_",
        "preload": "latest",
        "tokenizer_file": "tokenizer_{0}.json",
        "experiment_name": "runs/tmodel"
    }

config = get_config()

ds_raw = load_dataset(f"{config['datasource']}", f"{config['lang_src']}-{config['lang_tgt']}", split='train')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/28.1k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/5.73M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/32332 [00:00<?, ? examples/s]

In [8]:
f"{config['datasource']}", f"{config['lang_src']}-{config['lang_tgt']}"

('opus_books', 'en-it')

In [9]:
from datasets import load_dataset

# Load the IITB English-Hindi dataset
datahinset = load_dataset("cfilt/iitb-english-hindi")


README.md:   0%|          | 0.00/3.14k [00:00<?, ?B/s]

dataset_infos.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/190M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/85.7k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/500k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1659083 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/520 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2507 [00:00<?, ? examples/s]

In [10]:
# Load only the training split
thin = load_dataset("cfilt/iitb-english-hindi", split="train")


In [11]:
ds_raw

Dataset({
    features: ['id', 'translation'],
    num_rows: 32332
})

In [12]:
datahinset

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 1659083
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 520
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 2507
    })
})

In [13]:
thin

Dataset({
    features: ['translation'],
    num_rows: 1659083
})

In [16]:
thin.features

{'translation': {'en': Value(dtype='string', id=None),
  'hi': Value(dtype='string', id=None)}}

In [17]:
ds_raw.features

{'id': Value(dtype='string', id=None),
 'translation': Translation(languages=['en', 'it'], id=None)}

In [19]:
thin['translation'][:3]

[{'en': 'Give your application an accessibility workout',
  'hi': 'अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें'},
 {'en': 'Accerciser Accessibility Explorer',
  'hi': 'एक्सेर्साइसर पहुंचनीयता अन्वेषक'},
 {'en': 'The default plugin layout for the bottom panel',
  'hi': 'निचले पटल के लिए डिफोल्ट प्लग-इन खाका'}]

In [20]:
ds_raw['translation'][:3]

[{'en': 'Source: Project Gutenberg',
  'it': 'Source: www.liberliber.it/Audiobook available here'},
 {'en': 'Jane Eyre', 'it': 'Jane Eyre'},
 {'en': 'Charlotte Bronte', 'it': 'Charlotte Brontë'}]

In [26]:
from pathlib import Path
lang  = 'en'
tokenizer_path = Path(config['tokenizer_file'].format(lang))
tokenizer_path

PosixPath('tokenizer_en.json')

In [ ]:
def get_all_sentences(ds, lang):
    for item in ds:
        yield item['translation'][lang]

def get_or_build_tokenizer(config, ds, lang):
    tokenizer_path = Path(config['tokenizer_file'].format(lang))
    if not Path.exists(tokenizer_path):
        # Most code taken from: https://huggingface.co/docs/tokenizers/quicktour
        tokenizer = Tokenizer(WordLevel(unk_token="[UNK]"))
        tokenizer.pre_tokenizer = Whitespace()
        trainer = WordLevelTrainer(special_tokens=["[UNK]", "[PAD]", "[SOS]", "[EOS]"], min_frequency=2)
        tokenizer.train_from_iterator(get_all_sentences(ds, lang), trainer=trainer)
        tokenizer.save(str(tokenizer_path))
    else:
        tokenizer = Tokenizer.from_file(str(tokenizer_path))
    return tokenizer


# Build tokenizers
tokenizer_src = get_or_build_tokenizer(config, ds_raw, config['lang_src'])
tokenizer_tgt = get_or_build_tokenizer(config, ds_raw, config['lang_tgt'])

In addition to **word-level tokenizers**, there are several other types of tokenization models, each with different characteristics and trade-offs. Here's an overview of the common tokenizer models and how they differ:

### 1. **Word-Level Tokenizer**
   - **Description**: This tokenizer treats each word as a token, splitting text at spaces and punctuation marks.
   - **How It Works**: Words are the basic units of the vocabulary. Unknown or rare words are usually represented by an `[UNK]` token.
   - **Advantages**:
     - Simple to understand.
     - Vocabulary is easy to interpret (each token corresponds to a word).
   - **Disadvantages**:
     - Large vocabulary size (since every word is a token).
     - Handles rare or unknown words poorly (often mapped to `[UNK]`).
     - Difficult to handle out-of-vocabulary (OOV) words.
   - **Use Case**: Basic text processing where a smaller vocabulary is not a priority.

### 2. **Character-Level Tokenizer**
   - **Description**: Each individual character is treated as a token.
   - **How It Works**: The text is split into individual characters, so the vocabulary consists of all the possible characters (e.g., letters, numbers, punctuation).
   - **Advantages**:
     - Small vocabulary size (since it only needs to include characters).
     - Can handle any input text, including rare or unknown words.
   - **Disadvantages**:
     - Sequences are much longer (since words are split into multiple tokens).
     - Less semantic meaning per token, making training slower or less effective.
   - **Use Case**: Useful when dealing with languages with many possible word forms, such as agglutinative languages, or for tasks where handling any input without out-of-vocabulary issues is crucial.

### 3. **Subword Tokenizer**
   - **Description**: Breaks words into subword units based on frequency. Common words are kept intact, while rare or unknown words are split into smaller, more frequent subword tokens.
   - **How It Works**: Subword tokenization allows for a balance between word-level and character-level tokenization. It learns subword patterns from the data, so frequent word parts (like roots, prefixes, or suffixes) are common tokens, while rare words are broken into smaller parts.
   - **Types of Subword Tokenizers**:
     1. **Byte-Pair Encoding (BPE)**:
        - **How It Works**: Starts with individual characters as tokens, then iteratively merges the most frequent pairs of tokens into new tokens.
        - **Advantages**:
          - Reduces vocabulary size compared to word-level tokenization.
          - Efficient in capturing common subwords and word parts.
        - **Example**: "playing" → ["play", "ing"].
        - **Popular Use Case**: GPT models (like GPT-2, GPT-3), BERT.
      
     2. **WordPiece**:
        - **How It Works**: Similar to BPE but differs in the way it merges tokens based on likelihood in the context of language modeling.
        - **Advantages**:
          - Optimizes the choice of subword tokens based on the language model's context.
        - **Example**: "unhappiness" → ["un", "##happiness"] (subword pieces are indicated by "##").
        - **Popular Use Case**: BERT models.
      
     3. **Unigram Language Model**:
        - **How It Works**: Treats subwords as the base and selects the optimal vocabulary by evaluating the likelihood of various subword combinations.
        - **Advantages**:
          - More probabilistic, chooses the most likely segmentation for each word.
        - **Example**: "tokyo" → ["to", "kyo"].
        - **Popular Use Case**: SentencePiece (used in models like T5 and ALBERT).

   - **Advantages** of Subword Tokenizers:
     - Balance between vocabulary size and sequence length.
     - Can handle unseen or rare words by breaking them into familiar subword tokens.
     - Captures meaningful subword units (prefixes, suffixes, roots).
   - **Disadvantages**:
     - Training the tokenizer requires more computation compared to word-level tokenization.
     - The output tokens can be less intuitive than word-level tokens (e.g., breaking words into subword pieces).
   - **Use Case**: Most modern NLP models (e.g., GPT, BERT, T5) use subword tokenizers to balance vocabulary size and ability to handle rare words.

### 4. **SentencePiece Tokenizer**
   - **Description**: A subword tokenization method that can work without explicit whitespace splitting.
   - **How It Works**: Trains a tokenizer directly from raw text, without needing to split the text into words first. It is often used for languages without clear word boundaries, like Chinese and Japanese.
   - **Advantages**:
     - Does not rely on explicit whitespace-based tokenization.
     - Works well with languages that don’t use spaces between words.
   - **Disadvantages**:
     - Less intuitive for languages with clear word boundaries.
   - **Use Case**: Used in models like T5 and XLNet for multi-lingual text processing.

### 5. **Byte-Level BPE (Byte-Level Tokenizer)**
   - **Description**: This is a variant of Byte-Pair Encoding (BPE) that works on byte-level representations of text, rather than characters or words.
   - **How It Works**: Text is encoded as bytes, and BPE merges frequent byte pairs. This allows the tokenizer to work with any text encoding (like Unicode) and handle a wide variety of characters and symbols.
   - **Advantages**:
     - Can handle any input, including special characters and rare symbols.
     - Language-independent and avoids issues with unseen characters.
   - **Disadvantages**:
     - Sequences tend to be longer than word-level tokenizers.
   - **Use Case**: Used in models like GPT-2 and GPT-3 to process text at a byte level and capture rare symbols or characters.

### 6. **Sentence-Level Tokenizer**
   - **Description**: Splits the text into sentences as tokens.
   - **How It Works**: Text is split by sentence boundaries (e.g., based on punctuation like periods, question marks).
   - **Advantages**:
     - Useful for tasks where sentence boundaries matter (e.g., summarization, translation).
   - **Disadvantages**:
     - Not suitable for most word-level tasks.
   - **Use Case**: Text summarization, document-level tasks.

---

### Summary of Tokenizer Models:
| **Tokenizer Type**      | **Unit**                | **Vocabulary Size** | **Sequence Length**  | **Advantages**                              | **Disadvantages**                              | **Use Cases**                                   |
|-------------------------|-------------------------|---------------------|----------------------|---------------------------------------------|------------------------------------------------|-------------------------------------------------|
| **Word-Level**           | Words                   | Large               | Short                | Simple, interpretable                       | OOV issues, large vocabulary                   | Basic text processing                           |
| **Character-Level**      | Characters              | Small               | Long                 | Handles any text, small vocab               | Long sequences, less semantic meaning          | Character-level tasks, morphologically rich languages |
| **Subword (BPE)**        | Subword units (BPE)     | Moderate            | Moderate             | Balances vocab size and sequence length     | Can break words into unintuitive pieces        | GPT, BERT, general NLP tasks                   |
| **Subword (WordPiece)**  | Subword units           | Moderate            | Moderate             | Optimized for language models               | Similar to BPE, but more complex               | BERT                                            |
| **SentencePiece**        | Subword units (Unigram) | Moderate            | Moderate             | Doesn't need space-based tokenization       | Less intuitive for space-separated languages   | T5, ALBERT, languages without spaces            |
| **Byte-Level BPE**       | Bytes                   | Small               | Long                 | Handles any character or byte               | Long sequences, less semantic meaning          | GPT-2, GPT-3                                    |
| **Sentence-Level**       | Sentences               | Small               | Short                | Focuses on sentence-level structure         | Not suitable for word-level tasks              | Summarization, translation                      |

Each tokenizer model offers trade-offs between vocabulary size, sequence length, and the ability to handle out-of-vocabulary words or complex text inputs. **Subword tokenizers** (BPE, WordPiece) are the most widely used in modern NLP models because they offer a good balance between these factors.

The trade-offs between **vocabulary size** and **sequence length** in NLP tokenization arise due to how different tokenizers represent text. Larger vocabulary sizes tend to reduce sequence length, while smaller vocabulary sizes increase sequence length. Both have advantages and disadvantages, depending on the task and the model architecture. Let’s break down the trade-offs:

### 1. **Vocabulary Size**
   - **Larger Vocabulary**: In word-level tokenizers, each word is treated as a token, leading to a larger vocabulary.
     - **Pros**:
       - Fewer tokens are needed to represent a sentence (shorter sequences).
       - Captures more semantic information in each token (since tokens are full words).
       - Faster inference time due to shorter sequences.
     - **Cons**:
       - Large memory and storage requirements for the vocabulary.
       - Poor handling of out-of-vocabulary (OOV) words (since unseen words are mapped to an `[UNK]` token).
       - Inflexible for morphologically rich languages, where many word variations exist.
     
   - **Smaller Vocabulary**: In subword or character-level tokenizers, words are broken into smaller units (subwords or characters), reducing the vocabulary size.
     - **Pros**:
       - Handles unseen words better, as rare or OOV words can be constructed from known subword units.
       - Reduces memory and storage requirements (smaller vocabulary file).
       - More flexible for handling languages with complex morphology (e.g., agglutinative languages).
     - **Cons**:
       - Sequences are longer because each word is represented by multiple subword or character tokens.
       - Slower processing during both training and inference due to longer sequences.
       - Less semantic information in each token (a single subword or character carries less meaning than a whole word).

### 2. **Sequence Length**
   - **Shorter Sequences**: When using a larger vocabulary (e.g., word-level tokenization), sentences are represented by fewer tokens.
     - **Pros**:
       - Faster processing, especially in models like transformers where computational cost grows quadratically with sequence length.
       - Less memory usage during training and inference.
     - **Cons**:
       - Handling OOV words is problematic (many unseen words get mapped to `[UNK]`).
       - Inflexible when dealing with different morphological forms of words (like in agglutinative languages).
   
   - **Longer Sequences**: When using a smaller vocabulary (e.g., subword or character-level tokenization), each word may be split into multiple tokens, leading to longer sequences.
     - **Pros**:
       - Better handling of rare or unseen words by breaking them into known subword components.
       - More flexible across multiple languages and contexts, especially for handling complex languages.
     - **Cons**:
       - Increased computational cost, since sequence length directly affects the time and memory required for training and inference.
       - Training might be slower, especially for transformer-based models, due to the quadratic scaling of attention layers with respect to sequence length.

### Key Trade-offs:
| **Aspect**             | **Large Vocabulary (Short Sequences)**                                     | **Small Vocabulary (Long Sequences)**                                      |
|------------------------|---------------------------------------------------------------------------|---------------------------------------------------------------------------|
| **OOV Word Handling**   | Poor. OOV words become `[UNK]`, leading to loss of information.           | Excellent. Unseen words are broken into subword or character tokens.       |
| **Memory Usage**        | High memory usage due to large vocabulary.                                | Lower memory usage for storing a smaller vocabulary.                      |
| **Training Efficiency** | Faster, since fewer tokens per sentence mean shorter sequences.           | Slower, due to longer sequences.                                          |
| **Inference Speed**     | Faster, due to shorter sequences.                                         | Slower, due to longer sequences.                                          |
| **Representation Power**| High. Each token represents a full word, which retains more semantic meaning. | Lower. Subwords or characters represent smaller units of meaning.         |
| **Flexibility (Languages with Complex Morphology)** | Low. Words with different morphological variants may appear as separate tokens. | High. Morphological variations are represented through subwords or characters. |

### Example:
Let’s consider the word **"unhappiness"**:
- **Word-Level Tokenization** (Large Vocabulary, Short Sequence):  
  Vocabulary: ["happy", "unhappiness", "sad", ...]  
  Tokenized Output: `[unhappiness]` (Single token)  
  **Advantages**: Shorter sequence, full word meaning captured.  
  **Disadvantages**: If "unhappiness" wasn’t seen during training, it would be replaced by `[UNK]`.
  
- **Subword Tokenization (BPE or WordPiece)** (Medium Vocabulary, Moderate Sequence):  
  Vocabulary: ["un", "happiness", "happy", ...]  
  Tokenized Output: `[un, happiness]` (Two tokens)  
  **Advantages**: Handles unseen words better. If "unhappiness" wasn’t in the vocabulary, it could still be tokenized as `[un, happiness]`.  
  **Disadvantages**: Sequence is longer, though not drastically.

- **Character-Level Tokenization** (Small Vocabulary, Long Sequence):  
  Vocabulary: ['u', 'n', 'h', 'a', 'p', 'p', 'i', 'n', 'e', 's', 's']  
  Tokenized Output: `['u', 'n', 'h', 'a', 'p', 'p', 'i', 'n', 'e', 's', 's']` (11 tokens)  
  **Advantages**: Can handle any word, including rare or unseen words, since it’s broken into characters.  
  **Disadvantages**: Very long sequence, which increases computational cost and makes training less efficient.

### Conclusion:
The choice of vocabulary size and sequence length depends on your task:
- **Large Vocabulary (Short Sequence)** is better for tasks where speed and efficiency are crucial, and handling rare or unseen words is not a major concern (e.g., limited-domain tasks).
- **Small Vocabulary (Long Sequence)** is better for tasks where it's important to handle all possible words, especially in morphologically complex languages or for general-purpose models. However, it comes at the cost of increased computational requirements.

Modern NLP models, like **BERT** or **GPT**, typically use **subword tokenizers** (like BPE or WordPiece), as they strike a balance between vocabulary size and sequence length, making them flexible, efficient, and capable of handling unseen words well.

In [28]:
sentences = ["I love apples", "You love oranges"]


In [31]:
from datasets import load_dataset
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.trainers import WordLevelTrainer
from tokenizers.pre_tokenizers import Whitespace


tokenizer = Tokenizer(WordLevel(unk_token="[UNK]"))

tokenizer


In [32]:
trainer = WordLevelTrainer(
    special_tokens=["[UNK]", "[PAD]", "[SOS]", "[EOS]"],
    min_frequency=2
)


In [33]:
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.trainers import WordLevelTrainer
from tokenizers.pre_tokenizers import Whitespace
from pathlib import Path

# Sample sentences
sentences = ["I love apples", "You love oranges"]

# Function to simulate getting all sentences
def get_all_sentences(sentences):
    for sentence in sentences:
        yield sentence.split()  # Yield each sentence tokenized by words

# Initialize a WordLevel Tokenizer with [UNK] token
tokenizer = Tokenizer(WordLevel(unk_token="[UNK]"))

# Set the pre-tokenizer to split by whitespace
tokenizer.pre_tokenizer = Whitespace()

# Initialize the WordLevelTrainer with special tokens and minimum frequency
trainer = WordLevelTrainer(
    special_tokens=["[UNK]", "[PAD]", "[SOS]", "[EOS]"],
    min_frequency=2
)



In [35]:
for i in get_all_sentences(sentences):
  print(i)

['I', 'love', 'apples']
['You', 'love', 'oranges']


In [36]:
# Train the tokenizer using the sample sentences
tokenizer.train_from_iterator(get_all_sentences(sentences), trainer=trainer)

In [39]:
tokenizer.get_vocab()

{'[PAD]': 1, 'love': 4, '[SOS]': 2, '[EOS]': 3, '[UNK]': 0}

In [ ]:
# Train the tokenizer using the sample sentences
tokenizer.train_from_iterator(get_all_sentences(sentences), trainer=trainer)

# Save the tokenizer to a file
tokenizer_path = "wordlevel_tokenizer.json"
tokenizer.save(tokenizer_path)

# Test tokenization with the trained tokenizer
output = tokenizer.encode("I love apples")
print(f"Tokenized output for 'I love apples': {output.tokens}")

output = tokenizer.encode("You love oranges")
print(f"Tokenized output for 'You love oranges': {output.tokens}")

# Load the tokenizer again from the file
loaded_tokenizer = Tokenizer.from_file(tokenizer_path)

# Tokenize a sentence with the loaded tokenizer
new_output = loaded_tokenizer.encode("I love grapes")
print(f"Tokenized output for 'I love grapes': {new_output.tokens}")


Here's a step-by-step breakdown of how the tokenizer is trained using the code you provided, along with an example to illustrate the process:

### Steps:

1. **Initialize Tokenizer (WordLevel)**
   ```python
   tokenizer = Tokenizer(WordLevel(unk_token="[UNK]"))
   ```
   - This initializes a **WordLevel tokenizer**, which treats each word as a token.
   - If a word is not in the vocabulary during training or inference, it is replaced with the `[UNK]` (unknown) token.

   **Example**:  
   For an initial dataset containing sentences like `"I love apples"` and `"I hate oranges"`, words like `"love"` and `"hate"` might be added to the vocabulary. If an unseen word like `"grapes"` appears later, it would be replaced by `[UNK]`.

2. **Set Pre-Tokenizer (Whitespace)**
   ```python
   tokenizer.pre_tokenizer = Whitespace()
   ```
   - This sets the **pre-tokenizer** to split the input text into tokens using **whitespace**.
   - Each word (separated by spaces) will be treated as an individual token.

   **Example**:  
   Sentence: `"I love apples"`  
   Pre-tokenized as: `["I", "love", "apples"]`

3. **Set Trainer (WordLevelTrainer)**
   ```python
   trainer = WordLevelTrainer(
       special_tokens=["[UNK]", "[PAD]", "[SOS]", "[EOS]"],
       min_frequency=2
   )
   ```
   - The `WordLevelTrainer` object is used to **train the tokenizer**. It has two important parameters:
     - `special_tokens`: This list includes special tokens like `[UNK]` (unknown token), `[PAD]` (padding token), `[SOS]` (start of sequence), and `[EOS]` (end of sequence). These tokens will be reserved in the vocabulary.
     - `min_frequency=2`: Only words that appear **at least twice** in the dataset will be included in the tokenizer's vocabulary.

   **Example**:
   - If a word appears just once, it will not be included in the vocabulary, and the tokenizer will replace it with `[UNK]` during inference.

4. **Train Tokenizer on Dataset**
   ```python
   tokenizer.train_from_iterator(get_all_sentences(ds, lang), trainer=trainer)
   ```
   - The tokenizer is trained on the **dataset** by iterating over all sentences using `get_all_sentences(ds, lang)`.
   - The trainer builds a vocabulary from words in the dataset that meet the **minimum frequency** requirement (2 occurrences).

   **Example**:  
   Dataset:  
   ```text
   "I love apples"
   "I hate oranges"
   "You love apples"
   ```
   - Word Frequencies:
     - `"I"`: 3 occurrences
     - `"love"`: 2 occurrences
     - `"apples"`: 2 occurrences
     - `"hate"`: 1 occurrence
     - `"oranges"`: 1 occurrence
     - `"You"`: 1 occurrence
   - Vocabulary after training (considering `min_frequency=2`):
     - Vocabulary: `["I", "love", "apples", "[UNK]", "[PAD]", "[SOS]", "[EOS]"]`
     - Words like `"hate"`, `"oranges"`, and `"You"` are excluded from the vocabulary because they appear only once, so they will be replaced with `[UNK]` during inference.

5. **Save the Trained Tokenizer**
   ```python
   tokenizer.save(str(tokenizer_path))
   ```
   - After training, the tokenizer is **saved** to a specified file path for future use. This allows you to load the tokenizer without retraining it every time.

---

### **Small Example**: Training a Tokenizer

Imagine we have a dataset with two sentences:
```python
sentences = ["I love apples", "You love oranges"]
```

#### **Step-by-Step Tokenization Process:**

1. **Initialize Tokenizer:**
   ```python
   tokenizer = Tokenizer(WordLevel(unk_token="[UNK]"))
   ```

2. **Pre-tokenizer (Whitespace):**
   - Splits the sentences based on spaces.
   - `"I love apples"` → `["I", "love", "apples"]`
   - `"You love oranges"` → `["You", "love", "oranges"]`

3. **Set Trainer:**
   ```python
   trainer = WordLevelTrainer(
       special_tokens=["[UNK]", "[PAD]", "[SOS]", "[EOS]"],
       min_frequency=2
   )
   ```
   - Special tokens like `[UNK]`, `[PAD]`, `[SOS]`, and `[EOS]` will be included in the vocabulary, even if they don’t appear in the dataset.
   - Only words that appear at least twice (min_frequency=2) will be added to the vocabulary.

4. **Train Tokenizer:**
   - For the dataset, word frequencies are:
     - `"I"`: 1 occurrence
     - `"love"`: 2 occurrences
     - `"apples"`: 1 occurrence
     - `"You"`: 1 occurrence
     - `"oranges"`: 1 occurrence
   - Since `"love"` appears twice, it will be included in the vocabulary, but the other words will be excluded due to `min_frequency=2`.
   - Resulting vocabulary: `["love", "[UNK]", "[PAD]", "[SOS]", "[EOS]"]`

5. **Post-Training Tokenization:**
   - Now, if we tokenize a sentence:
     - `"I love apples"` → `["[UNK]", "love", "[UNK]"]` (because `"I"` and `"apples"` are not in the vocabulary).
   - Another example:
     - `"You love oranges"` → `["[UNK]", "love", "[UNK]"]` (because `"You"` and `"oranges"` are also not in the vocabulary).

6. **Save Tokenizer:**
   ```python
   tokenizer.save("path/to/tokenizer.json")
   ```

The tokenizer is saved to the path `"path/to/tokenizer.json"`, and you can load it in the future without retraining.

---

### Summary:
1. **Initialization**: A word-level tokenizer is created.
2. **Pre-tokenization**: Text is split into words by whitespace.
3. **Training**: A trainer object is created, specifying special tokens and a minimum frequency for including words in the vocabulary.
4. **Training from Dataset**: The tokenizer is trained on sentences from the dataset.
5. **Saving**: The tokenizer is saved to a file for future use.

This approach ensures that the tokenizer only includes relevant words based on the dataset and frequency threshold while handling unknown or rare words with the `[UNK]` token.

Building a BERT-like WordPiece tokenizer from scratch involves several steps. We'll create a simple version of a WordPiece tokenizer that trains on a small dataset, merges frequent subwords, and generates tokens based on the learned vocabulary.

### Step-by-Step Guide to Building a WordPiece Tokenizer

#### Step 1: Import Required Libraries
We'll use Python to build the tokenizer, and we may also use some basic libraries like `collections` for frequency counting.

```python
import re
from collections import defaultdict, Counter
```

#### Step 2: Define a Function to Tokenize Text into Characters
We'll start with a basic function that tokenizes input text into individual characters. This is the first step in building a subword tokenizer.

```python
def tokenize_text(text):
    return list(text)
```

#### Step 3: Build the Initial Vocabulary
We'll create a vocabulary based on the character tokens.

```python
def build_initial_vocab(sentences):
    vocab = Counter()
    for sentence in sentences:
        tokens = tokenize_text(sentence)
        vocab.update(tokens)
    return vocab
```

#### Step 4: Merge Frequent Character Pairs
Next, we will define a function that merges the most frequent pairs of tokens until we reach the desired vocabulary size.

```python
def get_stats(vocab):
    """Get statistics of token pairs."""
    pairs = defaultdict(int)
    for word, freq in vocab.items():
        symbols = word.split()
        for i in range(len(symbols) - 1):
            pairs[(symbols[i], symbols[i + 1])] += freq
    return pairs

def merge_vocab(pairs, vocab):
    """Merge the most frequent pair."""
    if not pairs:
        return vocab
    best_pair = max(pairs, key=pairs.get)
    new_token = ''.join(best_pair)
    new_vocab = Counter()
    for word, freq in vocab.items():
        new_word = word.replace(' '.join(best_pair), new_token)
        new_vocab[new_word] += freq
    return new_vocab
```

#### Step 5: Train the Tokenizer
Now, we will define a training function that will use the initial vocabulary to create a final vocabulary by merging character pairs.

```python
def train_wordpiece(sentences, vocab_size=30):
    # Build initial vocabulary
    vocab = build_initial_vocab(sentences)
    
    # Start with characters as tokens
    for i in range(vocab_size):
        pairs = get_stats(vocab)
        vocab = merge_vocab(pairs, vocab)
    
    return vocab
```

#### Step 6: Putting It All Together
We can now train our WordPiece tokenizer on a small set of example sentences.

```python
# Example sentences
sentences = [
    "I love apples",
    "You love oranges",
    "They play basketball",
    "We enjoy playing football"
]

# Train the WordPiece tokenizer
final_vocab = train_wordpiece(sentences, vocab_size=20)

# Display the final vocabulary
print("Final Vocabulary:")
for token, freq in final_vocab.items():
    print(f"{token}: {freq}")
```

### Output Example
After running the above code, you might see output like this:

```
Final Vocabulary:
a: 5
l: 8
e: 6
o: 7
p: 4
s: 5
i: 4
n: 3
y: 3
apples: 1
love: 2
basketball: 1
play: 2
football: 1
enjoy: 1
oranges: 1
They: 1
We: 1
You: 1
I: 1
```

### Explanation of Each Step:

1. **Tokenization**: The input sentences are broken down into individual characters to start the vocabulary.
   
2. **Initial Vocabulary**: We create an initial vocabulary of character counts using `Counter`, which helps in frequency counting.

3. **Pair Statistics**: For each word in the vocabulary, we gather statistics on the adjacent character pairs.

4. **Merging Pairs**: The most frequent character pairs are merged to create new tokens. This process is repeated until we reach the desired vocabulary size.

5. **Final Vocabulary**: The final vocabulary contains tokens that can represent both characters and merged subwords, which can be used for text input to a model.

### Conclusion:
This example demonstrates a simplified version of how a WordPiece tokenizer can be built from scratch. In practice, libraries like Hugging Face’s `transformers` provide optimized and pre-trained tokenizers, but understanding the basic mechanics behind tokenization helps grasp how models like BERT handle text.

Let’s break down the process of building a WordPiece tokenizer step by step using the example sentences:

```python
sentences = [
    "I love apples",
    "You love oranges",
    "They play basketball",
    "We enjoy playing football"
]
```

### Step 1: Tokenize Text into Characters

**Function**:
```python
def tokenize_text(text):
    return list(text)
```

**Explanation**:
- The tokenizer takes a string and converts it into a list of characters. For instance, the sentence "I love apples" will be tokenized into individual characters like this:
  - `"I love apples"` → `['I', ' ', 'l', 'o', 'v', 'e', ' ', 'a', 'p', 'p', 'l', 'e', 's']`

### Step 2: Build the Initial Vocabulary

**Function**:
```python
def build_initial_vocab(sentences):
    vocab = Counter()
    for sentence in sentences:
        tokens = tokenize_text(sentence)
        vocab.update(tokens)
    return vocab
```

**Explanation**:
- We initialize a `Counter` to keep track of how many times each character appears across all sentences.
- For each sentence, we tokenize it and update the frequency count.
- After processing all sentences, we get the initial character-level vocabulary. For the example sentences, the initial vocabulary might look like this:

```
{
    'I': 1,
    ' ': 13,  # space character
    'l': 6,
    'o': 5,
    'v': 3,
    'e': 5,
    'a': 5,
    'p': 4,
    's': 3,
    'y': 2,
    'n': 1,
    'g': 1,
    'b': 1,
    'k': 1,
    't': 1,
    'f': 1,
    'r': 1,
    'c': 1,
    'l': 1
}
```

### Step 3: Get Statistics of Token Pairs

**Function**:
```python
def get_stats(vocab):
    pairs = defaultdict(int)
    for word, freq in vocab.items():
        symbols = word.split()
        for i in range(len(symbols) - 1):
            pairs[(symbols[i], symbols[i + 1])] += freq
    return pairs
```

**Explanation**:
- This function computes the frequencies of adjacent character pairs in the current vocabulary.
- It looks for pairs of characters in each token and counts how often they occur together.
  
For example, considering the sentence "I love apples":
- The character pairs would be:
  - `('I', ' ')`, `(' ', 'l')`, `('l', 'o')`, `('o', 'v')`, `('v', 'e')`, etc.

### Step 4: Merge the Most Frequent Pair

**Function**:
```python
def merge_vocab(pairs, vocab):
    """Merge the most frequent pair."""
    if not pairs:
        return vocab
    best_pair = max(pairs, key=pairs.get)
    new_token = ''.join(best_pair)
    new_vocab = Counter()
    for word, freq in vocab.items():
        new_word = word.replace(' '.join(best_pair), new_token)
        new_vocab[new_word] += freq
    return new_vocab
```

**Explanation**:
- This function merges the most frequent pair of characters.
- It takes the pair with the highest frequency and replaces occurrences of that pair in the vocabulary with a new token.
- For example, if the pair `('l', 'o')` is the most frequent, it would create a new token `'lo'`.

### Step 5: Train the WordPiece Tokenizer

**Function**:
```python
def train_wordpiece(sentences, vocab_size=30):
    # Build initial vocabulary
    vocab = build_initial_vocab(sentences)
    
    # Start with characters as tokens
    for i in range(vocab_size):
        pairs = get_stats(vocab)
        vocab = merge_vocab(pairs, vocab)
    
    return vocab
```

**Explanation**:
- This function combines all previous steps to train the WordPiece tokenizer.
- It builds the initial character-level vocabulary and repeatedly merges pairs to expand the vocabulary up to the desired size.

### Example of the Entire Process

Let’s run through the training process step-by-step using our example sentences:

1. **Initial Vocabulary**:
   After tokenizing all sentences, we get a vocabulary that includes individual characters and their frequencies.

2. **First Merge**:
   - Calculate pairs from the initial vocabulary:
     - Most frequent pair might be `('l', 'o')` → `'lo'`.
   - Replace occurrences in the vocabulary.
  
3. **Subsequent Merges**:
   - Continue calculating pairs and merging them. For example:
     - Next pair might be `('a', 'p')` → `'ap'`.
   - Update vocabulary with the new tokens.
  
4. **Final Vocabulary**:
   After several iterations, we may end up with a vocabulary that includes:
   ```
   {
       'I': 1,
       ' ': 13,
       'lo': 2,
       've': 2,
       'ap': 3,
       'ples': 1,
       'you': 1,
       'ran': 1,
       'ge': 1,
       'basket': 1,
       'ball': 1,
       'enjoy': 1,
       'playing': 1,
       ...
   }
   ```

### Final Output
At the end of the training process, we have a vocabulary of merged tokens that can be used for text representation in models like BERT. Each token will now correspond to frequently occurring subwords or full words, allowing the model to handle a wide variety of inputs effectively.

### Conclusion
By building a tokenizer from scratch, we can see the importance of tokenization in natural language processing. WordPiece, as utilized in BERT, allows the model to understand and generate language more flexibly by handling both common and rare words efficiently.

In [42]:
from transformers import AutoTokenizer
from collections import defaultdict

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [43]:
tokenizer

BertTokenizerFast(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [44]:
corpus = [
    "This is the Hugging Face Course.",
    "This chapter is about tokenization.",
    "This section shows several tokenizer algorithms.",
    "Hopefully, you will be able to understand how they are trained and generate tokens.",
]

In [45]:
tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str("This is the Hugging Face Course.")

# this starts from index =0 and ends at index = 4

[('This', (0, 4)),
 ('is', (5, 7)),
 ('the', (8, 11)),
 ('Hugging', (12, 19)),
 ('Face', (20, 24)),
 ('Course', (25, 31)),
 ('.', (31, 32))]

In [47]:

from collections import defaultdict
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

corpus = [
    "This is the Hugging Face Course.",
    "This chapter is about tokenization.",
    "This section shows several tokenizer algorithms.",
    "Hopefully, you will be able to understand how they are trained and generate tokens.",
]

### get the frequency of each word ###
word_freqs = defaultdict(int)
for text in corpus:
    words_with_offsets = tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(text)
    new_words = [word for word, offset in words_with_offsets]
    print(new_words)
    for word in new_words:
        word_freqs[word] += 1

print(f"\nFinal Word Frequency: {word_freqs}")

['This', 'is', 'the', 'Hugging', 'Face', 'Course', '.']
['This', 'chapter', 'is', 'about', 'tokenization', '.']
['This', 'section', 'shows', 'several', 'tokenizer', 'algorithms', '.']
['Hopefully', ',', 'you', 'will', 'be', 'able', 'to', 'understand', 'how', 'they', 'are', 'trained', 'and', 'generate', 'tokens', '.']

Final Word Frequency: defaultdict(<class 'int'>, {'This': 3, 'is': 2, 'the': 1, 'Hugging': 1, 'Face': 1, 'Course': 1, '.': 4, 'chapter': 1, 'about': 1, 'tokenization': 1, 'section': 1, 'shows': 1, 'several': 1, 'tokenizer': 1, 'algorithms': 1, 'Hopefully': 1, ',': 1, 'you': 1, 'will': 1, 'be': 1, 'able': 1, 'to': 1, 'understand': 1, 'how': 1, 'they': 1, 'are': 1, 'trained': 1, 'and': 1, 'generate': 1, 'tokens': 1})


In [48]:
word_freqs

defaultdict(int,
            {'This': 3,
             'is': 2,
             'the': 1,
             'Hugging': 1,
             'Face': 1,
             'Course': 1,
             '.': 4,
             'chapter': 1,
             'about': 1,
             'tokenization': 1,
             'section': 1,
             'shows': 1,
             'several': 1,
             'tokenizer': 1,
             'algorithms': 1,
             'Hopefully': 1,
             ',': 1,
             'you': 1,
             'will': 1,
             'be': 1,
             'able': 1,
             'to': 1,
             'understand': 1,
             'how': 1,
             'they': 1,
             'are': 1,
             'trained': 1,
             'and': 1,
             'generate': 1,
             'tokens': 1})

In [49]:

### split all word into alphabet ###
alphabet = []
for word in word_freqs.keys():
    if word[0] not in alphabet:
        alphabet.append(word[0])
    for letter in word[1:]:
        if f"##{letter}" not in alphabet:
            alphabet.append(f"##{letter}")

alphabet.sort()
print(f'All alphabets: {alphabet}')

### insert special token and subword ###
vocab = ["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"] + alphabet.copy()
splits = {word: [c if i == 0 else f"##{c}" for i, c in enumerate(word)] for word in word_freqs.keys()}
print(f'\nSplitted Words: {splits}')

All alphabets: ['##a', '##b', '##c', '##d', '##e', '##f', '##g', '##h', '##i', '##k', '##l', '##m', '##n', '##o', '##p', '##r', '##s', '##t', '##u', '##v', '##w', '##y', '##z', ',', '.', 'C', 'F', 'H', 'T', 'a', 'b', 'c', 'g', 'h', 'i', 's', 't', 'u', 'w', 'y']

Splitted Words: {'This': ['T', '##h', '##i', '##s'], 'is': ['i', '##s'], 'the': ['t', '##h', '##e'], 'Hugging': ['H', '##u', '##g', '##g', '##i', '##n', '##g'], 'Face': ['F', '##a', '##c', '##e'], 'Course': ['C', '##o', '##u', '##r', '##s', '##e'], '.': ['.'], 'chapter': ['c', '##h', '##a', '##p', '##t', '##e', '##r'], 'about': ['a', '##b', '##o', '##u', '##t'], 'tokenization': ['t', '##o', '##k', '##e', '##n', '##i', '##z', '##a', '##t', '##i', '##o', '##n'], 'section': ['s', '##e', '##c', '##t', '##i', '##o', '##n'], 'shows': ['s', '##h', '##o', '##w', '##s'], 'several': ['s', '##e', '##v', '##e', '##r', '##a', '##l'], 'tokenizer': ['t', '##o', '##k', '##e', '##n', '##i', '##z', '##e', '##r'], 'algorithms': ['a', '##l', '##g'

In [50]:
splits

{'This': ['T', '##h', '##i', '##s'],
 'is': ['i', '##s'],
 'the': ['t', '##h', '##e'],
 'Hugging': ['H', '##u', '##g', '##g', '##i', '##n', '##g'],
 'Face': ['F', '##a', '##c', '##e'],
 'Course': ['C', '##o', '##u', '##r', '##s', '##e'],
 '.': ['.'],
 'chapter': ['c', '##h', '##a', '##p', '##t', '##e', '##r'],
 'about': ['a', '##b', '##o', '##u', '##t'],
 'tokenization': ['t',
  '##o',
  '##k',
  '##e',
  '##n',
  '##i',
  '##z',
  '##a',
  '##t',
  '##i',
  '##o',
  '##n'],
 'section': ['s', '##e', '##c', '##t', '##i', '##o', '##n'],
 'shows': ['s', '##h', '##o', '##w', '##s'],
 'several': ['s', '##e', '##v', '##e', '##r', '##a', '##l'],
 'tokenizer': ['t', '##o', '##k', '##e', '##n', '##i', '##z', '##e', '##r'],
 'algorithms': ['a',
  '##l',
  '##g',
  '##o',
  '##r',
  '##i',
  '##t',
  '##h',
  '##m',
  '##s'],
 'Hopefully': ['H', '##o', '##p', '##e', '##f', '##u', '##l', '##l', '##y'],
 ',': [','],
 'you': ['y', '##o', '##u'],
 'will': ['w', '##i', '##l', '##l'],
 'be': ['b', '##e

In [51]:
vocab

['[PAD]',
 '[UNK]',
 '[CLS]',
 '[SEP]',
 '[MASK]',
 '##a',
 '##b',
 '##c',
 '##d',
 '##e',
 '##f',
 '##g',
 '##h',
 '##i',
 '##k',
 '##l',
 '##m',
 '##n',
 '##o',
 '##p',
 '##r',
 '##s',
 '##t',
 '##u',
 '##v',
 '##w',
 '##y',
 '##z',
 ',',
 '.',
 'C',
 'F',
 'H',
 'T',
 'a',
 'b',
 'c',
 'g',
 'h',
 'i',
 's',
 't',
 'u',
 'w',
 'y']

In [52]:
alphabet

['##a',
 '##b',
 '##c',
 '##d',
 '##e',
 '##f',
 '##g',
 '##h',
 '##i',
 '##k',
 '##l',
 '##m',
 '##n',
 '##o',
 '##p',
 '##r',
 '##s',
 '##t',
 '##u',
 '##v',
 '##w',
 '##y',
 '##z',
 ',',
 '.',
 'C',
 'F',
 'H',
 'T',
 'a',
 'b',
 'c',
 'g',
 'h',
 'i',
 's',
 't',
 'u',
 'w',
 'y']

In [54]:
len(vocab), len(alphabet  )

(45, 40)

In [55]:
 ### compute score for merging ###
def compute_pair_scores(splits):
    letter_freqs = defaultdict(int)
    pair_freqs = defaultdict(int)

    for word, freq in word_freqs.items():
        split = splits[word]
        if len(split) == 1:
            letter_freqs[split[0]] += freq
            continue
        for i in range(len(split) - 1):
            pair = (split[i], split[i + 1])
            letter_freqs[split[i]] += freq
            pair_freqs[pair] += freq
        letter_freqs[split[-1]] += freq

    scores = {
        pair: freq / (letter_freqs[pair[0]] * letter_freqs[pair[1]])
        for pair, freq in pair_freqs.items()
    }
    return scores

pair_scores = compute_pair_scores(splits)
print(f'Scores for each Pair: {pair_scores}')

Scores for each Pair: {('T', '##h'): 0.125, ('##h', '##i'): 0.03409090909090909, ('##i', '##s'): 0.02727272727272727, ('i', '##s'): 0.1, ('t', '##h'): 0.03571428571428571, ('##h', '##e'): 0.011904761904761904, ('H', '##u'): 0.1, ('##u', '##g'): 0.05, ('##g', '##g'): 0.0625, ('##g', '##i'): 0.022727272727272728, ('##i', '##n'): 0.01652892561983471, ('##n', '##g'): 0.022727272727272728, ('F', '##a'): 0.14285714285714285, ('##a', '##c'): 0.07142857142857142, ('##c', '##e'): 0.023809523809523808, ('C', '##o'): 0.07692307692307693, ('##o', '##u'): 0.046153846153846156, ('##u', '##r'): 0.022222222222222223, ('##r', '##s'): 0.022222222222222223, ('##s', '##e'): 0.004761904761904762, ('c', '##h'): 0.125, ('##h', '##a'): 0.017857142857142856, ('##a', '##p'): 0.07142857142857142, ('##p', '##t'): 0.07142857142857142, ('##t', '##e'): 0.013605442176870748, ('##e', '##r'): 0.026455026455026454, ('a', '##b'): 0.2, ('##b', '##o'): 0.038461538461538464, ('##u', '##t'): 0.02857142857142857, ('t', '##o')

In [56]:
pair_scores

{('T', '##h'): 0.125,
 ('##h', '##i'): 0.03409090909090909,
 ('##i', '##s'): 0.02727272727272727,
 ('i', '##s'): 0.1,
 ('t', '##h'): 0.03571428571428571,
 ('##h', '##e'): 0.011904761904761904,
 ('H', '##u'): 0.1,
 ('##u', '##g'): 0.05,
 ('##g', '##g'): 0.0625,
 ('##g', '##i'): 0.022727272727272728,
 ('##i', '##n'): 0.01652892561983471,
 ('##n', '##g'): 0.022727272727272728,
 ('F', '##a'): 0.14285714285714285,
 ('##a', '##c'): 0.07142857142857142,
 ('##c', '##e'): 0.023809523809523808,
 ('C', '##o'): 0.07692307692307693,
 ('##o', '##u'): 0.046153846153846156,
 ('##u', '##r'): 0.022222222222222223,
 ('##r', '##s'): 0.022222222222222223,
 ('##s', '##e'): 0.004761904761904762,
 ('c', '##h'): 0.125,
 ('##h', '##a'): 0.017857142857142856,
 ('##a', '##p'): 0.07142857142857142,
 ('##p', '##t'): 0.07142857142857142,
 ('##t', '##e'): 0.013605442176870748,
 ('##e', '##r'): 0.026455026455026454,
 ('a', '##b'): 0.2,
 ('##b', '##o'): 0.038461538461538464,
 ('##u', '##t'): 0.02857142857142857,
 ('t',

In [57]:
splits

{'This': ['T', '##h', '##i', '##s'],
 'is': ['i', '##s'],
 'the': ['t', '##h', '##e'],
 'Hugging': ['H', '##u', '##g', '##g', '##i', '##n', '##g'],
 'Face': ['F', '##a', '##c', '##e'],
 'Course': ['C', '##o', '##u', '##r', '##s', '##e'],
 '.': ['.'],
 'chapter': ['c', '##h', '##a', '##p', '##t', '##e', '##r'],
 'about': ['a', '##b', '##o', '##u', '##t'],
 'tokenization': ['t',
  '##o',
  '##k',
  '##e',
  '##n',
  '##i',
  '##z',
  '##a',
  '##t',
  '##i',
  '##o',
  '##n'],
 'section': ['s', '##e', '##c', '##t', '##i', '##o', '##n'],
 'shows': ['s', '##h', '##o', '##w', '##s'],
 'several': ['s', '##e', '##v', '##e', '##r', '##a', '##l'],
 'tokenizer': ['t', '##o', '##k', '##e', '##n', '##i', '##z', '##e', '##r'],
 'algorithms': ['a',
  '##l',
  '##g',
  '##o',
  '##r',
  '##i',
  '##t',
  '##h',
  '##m',
  '##s'],
 'Hopefully': ['H', '##o', '##p', '##e', '##f', '##u', '##l', '##l', '##y'],
 ',': [','],
 'you': ['y', '##o', '##u'],
 'will': ['w', '##i', '##l', '##l'],
 'be': ['b', '##e

In [58]:
### finding pair with best score ###
best_pair = ""
max_score = None
for pair, score in pair_scores.items():
    if max_score is None or max_score < score:
        best_pair = pair
        max_score = score

print(best_pair, max_score)

('a', '##b') 0.2


In [59]:
best_pair

('a', '##b')

In [60]:
max_score

0.2

In [61]:
### merge pair ###
def merge_pair(a, b, splits):
    for word in word_freqs:
        split = splits[word]
        if len(split) == 1:
            continue
        i = 0
        while i < len(split) - 1:
            if split[i] == a and split[i + 1] == b:
                merge = a + b[2:] if b.startswith("##") else a + b
                split = split[:i] + [merge] + split[i + 2 :]
            else:
                i += 1
        splits[word] = split
    return splits

splits = merge_pair("a", "##b", splits)
print(splits["about"])

['ab', '##o', '##u', '##t']


In [62]:
splits

{'This': ['T', '##h', '##i', '##s'],
 'is': ['i', '##s'],
 'the': ['t', '##h', '##e'],
 'Hugging': ['H', '##u', '##g', '##g', '##i', '##n', '##g'],
 'Face': ['F', '##a', '##c', '##e'],
 'Course': ['C', '##o', '##u', '##r', '##s', '##e'],
 '.': ['.'],
 'chapter': ['c', '##h', '##a', '##p', '##t', '##e', '##r'],
 'about': ['ab', '##o', '##u', '##t'],
 'tokenization': ['t',
  '##o',
  '##k',
  '##e',
  '##n',
  '##i',
  '##z',
  '##a',
  '##t',
  '##i',
  '##o',
  '##n'],
 'section': ['s', '##e', '##c', '##t', '##i', '##o', '##n'],
 'shows': ['s', '##h', '##o', '##w', '##s'],
 'several': ['s', '##e', '##v', '##e', '##r', '##a', '##l'],
 'tokenizer': ['t', '##o', '##k', '##e', '##n', '##i', '##z', '##e', '##r'],
 'algorithms': ['a',
  '##l',
  '##g',
  '##o',
  '##r',
  '##i',
  '##t',
  '##h',
  '##m',
  '##s'],
 'Hopefully': ['H', '##o', '##p', '##e', '##f', '##u', '##l', '##l', '##y'],
 ',': [','],
 'you': ['y', '##o', '##u'],
 'will': ['w', '##i', '##l', '##l'],
 'be': ['b', '##e'],
 '

In [63]:
vocab

['[PAD]',
 '[UNK]',
 '[CLS]',
 '[SEP]',
 '[MASK]',
 '##a',
 '##b',
 '##c',
 '##d',
 '##e',
 '##f',
 '##g',
 '##h',
 '##i',
 '##k',
 '##l',
 '##m',
 '##n',
 '##o',
 '##p',
 '##r',
 '##s',
 '##t',
 '##u',
 '##v',
 '##w',
 '##y',
 '##z',
 ',',
 '.',
 'C',
 'F',
 'H',
 'T',
 'a',
 'b',
 'c',
 'g',
 'h',
 'i',
 's',
 't',
 'u',
 'w',
 'y']

In [64]:
len(vocab)

45

In [65]:
### keep looping to merge more pair
vocab_size = 70
while len(vocab) < vocab_size:
    scores = compute_pair_scores(splits)
    best_pair, max_score = "", None
    for pair, score in scores.items():
        if max_score is None or max_score < score:
            best_pair = pair
            max_score = score
    splits = merge_pair(*best_pair, splits)
    new_token = (
        best_pair[0] + best_pair[1][2:]
        if best_pair[1].startswith("##")
        else best_pair[0] + best_pair[1]
    )
    vocab.append(new_token)

print(f'Final Vocab: {vocab}')

Final Vocab: ['[PAD]', '[UNK]', '[CLS]', '[SEP]', '[MASK]', '##a', '##b', '##c', '##d', '##e', '##f', '##g', '##h', '##i', '##k', '##l', '##m', '##n', '##o', '##p', '##r', '##s', '##t', '##u', '##v', '##w', '##y', '##z', ',', '.', 'C', 'F', 'H', 'T', 'a', 'b', 'c', 'g', 'h', 'i', 's', 't', 'u', 'w', 'y', '##fu', 'Fa', 'Fac', '##ct', '##ful', '##full', '##fully', 'Th', 'ch', '##hm', 'cha', 'chap', 'chapt', '##thm', 'Hu', 'Hug', 'Hugg', 'sh', 'th', 'is', '##thms', '##za', '##zat', '##ut', '##ta']


In [66]:
vocab

['[PAD]',
 '[UNK]',
 '[CLS]',
 '[SEP]',
 '[MASK]',
 '##a',
 '##b',
 '##c',
 '##d',
 '##e',
 '##f',
 '##g',
 '##h',
 '##i',
 '##k',
 '##l',
 '##m',
 '##n',
 '##o',
 '##p',
 '##r',
 '##s',
 '##t',
 '##u',
 '##v',
 '##w',
 '##y',
 '##z',
 ',',
 '.',
 'C',
 'F',
 'H',
 'T',
 'a',
 'b',
 'c',
 'g',
 'h',
 'i',
 's',
 't',
 'u',
 'w',
 'y',
 '##fu',
 'Fa',
 'Fac',
 '##ct',
 '##ful',
 '##full',
 '##fully',
 'Th',
 'ch',
 '##hm',
 'cha',
 'chap',
 'chapt',
 '##thm',
 'Hu',
 'Hug',
 'Hugg',
 'sh',
 'th',
 'is',
 '##thms',
 '##za',
 '##zat',
 '##ut',
 '##ta']

In [67]:
len(vocab)

70

In [68]:
### ro encode a word ###
def encode_word(word):
    tokens = []
    while len(word) > 0:
        i = len(word)
        while i > 0 and word[:i] not in vocab:
            i -= 1
        if i == 0:
            return ["[UNK]"]
        tokens.append(word[:i])
        word = word[i:]
        if len(word) > 0:
            word = f"##{word}"
    return tokens

print(encode_word("Hugging"))
print(encode_word("HOgging"))

['Hugg', '##i', '##n', '##g']
['[UNK]']


The output of the code `tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str("This is the Hugging Face Course.")` gives you a detailed breakdown of how the text is pre-tokenized. Let's break down the output step by step:

### Breakdown of the Output

The output is a list of tuples, where each tuple represents a token and its corresponding position in the original string. Here’s the format of each tuple:

- **Token**: The word or punctuation mark identified as a token.
- **Position**: A tuple containing two integers:
  - The starting index of the token in the original string.
  - The ending index of the token in the original string (exclusive).

### Example Output Explained

The output is as follows:

```python
[
    ('This', (0, 4)),
    ('is', (5, 7)),
    ('the', (8, 11)),
    ('Hugging', (12, 19)),
    ('Face', (20, 24)),
    ('Course', (25, 31)),
    ('.', (31, 32))
]
```

1. **Token: 'This'**  
   - **Position**: (0, 4)  
   - This means that the word "This" starts at index 0 and ends at index 4 in the original string.

2. **Token: 'is'**  
   - **Position**: (5, 7)  
   - The word "is" starts at index 5 and ends at index 7.

3. **Token: 'the'**  
   - **Position**: (8, 11)  
   - The word "the" starts at index 8 and ends at index 11.

4. **Token: 'Hugging'**  
   - **Position**: (12, 19)  
   - The word "Hugging" starts at index 12 and ends at index 19.

5. **Token: 'Face'**  
   - **Position**: (20, 24)  
   - The word "Face" starts at index 20 and ends at index 24.

6. **Token: 'Course'**  
   - **Position**: (25, 31)  
   - The word "Course" starts at index 25 and ends at index 31.

7. **Token: '.'**  
   - **Position**: (31, 32)  
   - The punctuation "." starts at index 31 and ends at index 32.

### Significance of the Output

This output is crucial for understanding how the tokenizer preprocesses the input text:

- **Pre-tokenization**: The step performed by the pre-tokenizer divides the input string into tokens based on whitespace and possibly other criteria (in this case, it simply splits by spaces and punctuation).
- **Index Tracking**: The positions allow you to see exactly where each token is located in the original input. This can be useful for tasks where you need to map tokens back to their original locations in the text, such as in Named Entity Recognition (NER) or when generating token-level annotations.

### Conclusion

The output you received is a clear representation of how the tokenizer processes input strings, identifying individual tokens and their respective positions, which is essential for any further tokenization, encoding, or NLP tasks.

Sure! Let’s break down the code you provided into its components and explain the steps involved, focusing on pre-tokenization and splitting into individual characters.

### Pre-tokenization
```python
word_freqs = defaultdict(int)
for text in corpus:
    words_with_offsets = tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(text)
    new_words = [word for word, offset in words_with_offsets]
    for word in new_words:
        word_freqs[word] += 1

print(word_freqs)
```

#### Explanation:

1. **Initialize a Frequency Dictionary**:
   - `word_freqs = defaultdict(int)` initializes a dictionary that will hold the frequency of each word. The `defaultdict(int)` automatically initializes the count to 0 for any new word.

2. **Loop Through the Corpus**:
   - `for text in corpus:` iterates through each text in a list (or any iterable) called `corpus`. This `corpus` is assumed to be a collection of text samples (e.g., sentences or paragraphs).

3. **Pre-tokenization**:
   - `words_with_offsets = tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(text)` calls the pre-tokenizer to break down the current text into tokens (words) and their offsets (the start and end positions of each token in the original text). The output is a list of tuples where each tuple contains a token and its corresponding position.

4. **Extract Words**:
   - `new_words = [word for word, offset in words_with_offsets]` creates a new list of just the words (ignoring the offsets) from the tuples returned by the pre-tokenizer.

5. **Count Word Frequencies**:
   - `for word in new_words: word_freqs[word] += 1` iterates through the extracted words and updates the frequency count in `word_freqs`. If a word appears, its count is incremented by 1.

6. **Print the Word Frequencies**:
   - `print(word_freqs)` displays the frequency of each word found in the entire corpus.

### Example Output of Word Frequencies
If your corpus contains sentences like:
- "I love apples."
- "You love oranges."

The output might look something like:
```python
defaultdict(<class 'int'>, {'I': 1, 'love': 2, 'apples': 1, 'You': 1, 'oranges': 1, '.': 2})
```
This indicates that the word "love" appears 2 times, while "I", "apples", "You", and "oranges" each appear once.

---

### Splitting Into Single Alphabets
```python
alphabet = []
for word in word_freqs.keys():
    for letter in word:
        if letter not in alphabet:
            alphabet.append(letter)
alphabet.sort()
print(alphabet)
```

#### Explanation:

1. **Initialize the Alphabet List**:
   - `alphabet = []` initializes an empty list that will store unique letters (characters) found in the words.

2. **Iterate Through Each Word**:
   - `for word in word_freqs.keys():` loops through each unique word in the `word_freqs` dictionary.

3. **Extract Letters**:
   - `for letter in word:` iterates through each character in the current word.

4. **Add Unique Letters**:
   - `if letter not in alphabet:` checks if the letter is not already in the `alphabet` list. If it’s a new letter, `alphabet.append(letter)` adds it to the list.

5. **Sort the Alphabet**:
   - `alphabet.sort()` sorts the list of unique letters in alphabetical order.

6. **Print the Unique Letters**:
   - `print(alphabet)` displays the sorted list of unique letters.

### Example Output of Unique Letters
For example, if your corpus contained the words:
- "I love apples."
- "You love oranges."

The output might look like:
```python
['.', 'I', 'Y', 'a', 'e', 'l', 'n', 'o', 'p', 'r', 's', 'u', 'v']
```
This indicates that the unique letters, including punctuation, from the words are displayed in sorted order.

### Summary
- The first block of code (`pre-tokenization`) counts the occurrences of each word in a given corpus, providing a frequency distribution of words.
- The second block of code extracts all unique letters from the counted words, allowing you to see which characters appear in the corpus text.

Together, these steps are foundational for building a vocabulary or for character-level processing in Natural Language Processing (NLP) tasks.

Let’s take a small example sentence and walk through the code step-by-step to understand how pre-tokenization and splitting into single alphabets work.

### Example Sentence
Let's use the sentence:  
**"I love apples."**

### Step 1: Pre-tokenization
**Code Snippet:**
```python
word_freqs = defaultdict(int)
text = "I love apples."
words_with_offsets = tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(text)
new_words = [word for word, offset in words_with_offsets]
for word in new_words:
    word_freqs[word] += 1

print(word_freqs)
```

#### Breakdown:

1. **Initialize a Frequency Dictionary**:
   - We create a `defaultdict` to hold the frequency of each word.
   - ```python
     word_freqs = defaultdict(int)
     ```

2. **Text Input**:
   - The input text is `"I love apples."`.

3. **Pre-tokenization**:
   - Assume the pre-tokenizer splits the sentence into tokens (words) and their respective offsets.
   - For our example, the output of `pre_tokenize_str` might look like this:
     ```python
     words_with_offsets = [
         ('I', (0, 1)),
         ('love', (2, 6)),
         ('apples', (7, 13)),
         ('.', (13, 14))
     ]
     ```

4. **Extract Words**:
   - We create a list of just the words from the tuples.
   - ```python
     new_words = ['I', 'love', 'apples', '.']
     ```

5. **Count Word Frequencies**:
   - Loop through the `new_words` list and update the word frequencies:
     - For `'I'`, count becomes 1.
     - For `'love'`, count becomes 1.
     - For `'apples'`, count becomes 1.
     - For `'.'`, count becomes 1.
   - The final `word_freqs` dictionary will look like:
     ```python
     defaultdict(<class 'int'>, {'I': 1, 'love': 1, 'apples': 1, '.': 1})
     ```

### Step 2: Splitting Into Single Alphabets
**Code Snippet:**
```python
alphabet = []
for word in word_freqs.keys():
    for letter in word:
        if letter not in alphabet:
            alphabet.append(letter)
alphabet.sort()
print(alphabet)
```

#### Breakdown:

1. **Initialize the Alphabet List**:
   - We create an empty list to store unique letters.
   - ```python
     alphabet = []
     ```

2. **Iterate Through Each Word**:
   - The `word_freqs` keys are `['I', 'love', 'apples', '.']`.

3. **Extract Letters**:
   - Loop through each word:
     - For `'I'`, add `'I'` to `alphabet`.
     - For `'love'`, add `'l'`, `'o'`, `'v'`, and `'e'` to `alphabet`.
     - For `'apples'`, add `'a'`, `'p'`, `'l'`, `'e'`, and `'s'` (note: `'l'` is already in the list).
     - For `'.'`, add `'.'` to `alphabet`.

4. **Add Unique Letters**:
   - After processing all words, the `alphabet` list will look like:
     ```python
     ['I', 'a', 'e', 'l', 'love', 'o', 'p', 's', 'v', '.']
     ```

5. **Sort the Alphabet**:
   - Sorting the `alphabet` gives:
     ```python
     ['.', 'I', 'a', 'e', 'l', 'l', 'o', 'p', 's', 'v']
     ```

6. **Print the Unique Letters**:
   - The final output for unique letters is:
     ```python
     print(alphabet)
     ```
   - Output:
     ```python
     ['.', 'I', 'a', 'e', 'l', 'o', 'p', 's', 'v']
     ```

### Summary of Steps for the Example Sentence
1. **Pre-tokenization**:
   - The sentence is broken down into tokens (words) along with their positions.
   - Word frequencies are counted, resulting in a dictionary that shows how many times each word appears.

2. **Splitting into Single Alphabets**:
   - The unique letters from all words are extracted, avoiding duplicates.
   - The letters are sorted, resulting in a list of unique characters used in the sentence.

This example illustrates the entire process from input text to generating a frequency dictionary of words and a sorted list of unique letters, showing how pre-tokenization and character extraction work in practice.

Let's break down the code that inserts special tokens and subword representations into the vocabulary based on the previously calculated alphabet and word frequencies.

### Code Explanation

**Code Snippet:**
```python
### insert special token and subword ###
vocab = ["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"] + alphabet.copy()
splits = {word: [c if i == 0 else f"##{c}" for i, c in enumerate(word)] for word in word_freqs.keys()}
print(f'\nSplitted Words: {splits}')
```

### Breakdown of Steps

1. **Create a Vocabulary List**:
   - The `vocab` list is initialized with special tokens used in BERT-like models:
     - `[PAD]`: Padding token, used to make all sequences in a batch the same length.
     - `[UNK]`: Unknown token, represents words not found in the vocabulary.
     - `[CLS]`: Classification token, used at the beginning of the input sequence for classification tasks.
     - `[SEP]`: Separator token, used to separate different segments in the input.
     - `[MASK]`: Mask token, used for masked language modeling tasks.
   - After adding these special tokens, we append a copy of the `alphabet` list, which contains the individual characters and their subword representations:
   ```python
   vocab = ["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"] + alphabet.copy()
   ```

2. **Generate Subword Representations**:
   - The `splits` dictionary is created using a dictionary comprehension:
   - For each word in `word_freqs.keys()`, we generate a list that contains:
     - The first character of the word as is (without any prefix).
     - Each subsequent character prefixed with `##`.
   - This is done using `enumerate(word)` to check the index (`i`) of each character:
   ```python
   splits = {word: [c if i == 0 else f"##{c}" for i, c in enumerate(word)] for word in word_freqs.keys()}
   ```

#### Example Breakdown
Given our example with `word_freqs`:
```python
word_freqs = defaultdict(int, {'I': 1, 'love': 1, 'apples': 1, 'basketball': 1})
```

#### Process Each Word:
1. **Word: 'I'**
   - `splits['I']` becomes:
   ```python
   ['I']  # Only one character, so no subwords.
   ```

2. **Word: 'love'**
   - `splits['love']` becomes:
   ```python
   ['l', '##o', '##v', '##e']
   ```

3. **Word: 'apples'**
   - `splits['apples']` becomes:
   ```python
   ['a', '##p', '##p', '##l', '##e', '##s']
   ```

4. **Word: 'basketball'**
   - `splits['basketball']` becomes:
   ```python
   ['b', '##a', '##s', '##k', '##e', '##t', '##b', '##a', '##l', '##l']
   ```

### Final Output
After running the code, we can print the `splits` dictionary to see how each word has been broken down into characters and subwords:
```python
print(f'\nSplitted Words: {splits}')
```

### Example Output
```plaintext
Splitted Words: {
    'I': ['I'],
    'love': ['l', '##o', '##v', '##e'],
    'apples': ['a', '##p', '##p', '##l', '##e', '##s'],
    'basketball': ['b', '##a', '##s', '##k', '##e', '##t', '##b', '##a', '##l', '##l']
}
```

### Summary
1. **Vocabulary Initialization**: The `vocab` list contains special tokens and the alphabet.
2. **Subword Generation**: The `splits` dictionary contains each word from `word_freqs`, where the first character is unmodified, and subsequent characters are prefixed with `##`, preparing them for subword tokenization as used in models like BERT.
3. **Final Output**: The resulting `splits` dictionary demonstrates how each word can be represented in a tokenized format suitable for model input.

This process is a key part of creating a tokenizer that can handle words flexibly, allowing it to break down unknown words into known subwords, enhancing the model's ability to understand and generate text.

Let's break down the provided code that computes scores for merging pairs of letters or subwords based on their frequencies in a given corpus. This process is commonly used in tokenization methods like Byte Pair Encoding (BPE), which are essential for creating vocabulary for models like BERT.

### Code Explanation

**Code Snippet:**
```python
### compute score for merging ###
def compute_pair_scores(splits):
    letter_freqs = defaultdict(int)
    pair_freqs = defaultdict(int)

    for word, freq in word_freqs.items():
        split = splits[word]
        if len(split) == 1:
            letter_freqs[split[0]] += freq
            continue
        for i in range(len(split) - 1):
            pair = (split[i], split[i + 1])
            letter_freqs[split[i]] += freq
            pair_freqs[pair] += freq
        letter_freqs[split[-1]] += freq

    scores = {
        pair: freq / (letter_freqs[pair[0]] * letter_freqs[pair[1]])
        for pair, freq in pair_freqs.items()
    }
    return scores

pair_scores = compute_pair_scores(splits)
print(f'Scores for each Pair: {pair_scores}')
```

### Breakdown of Steps

1. **Initialization of Frequency Counters**:
   - `letter_freqs`: A dictionary to count the frequency of each letter (or subword).
   - `pair_freqs`: A dictionary to count the frequency of each adjacent pair of letters.

2. **Iterate Over Each Word**:
   - The code loops through each `word` and its corresponding `freq` (frequency) in the `word_freqs` dictionary (which is assumed to have been calculated previously).
   - For each word, the corresponding split representation from the `splits` dictionary is retrieved.

3. **Counting Frequencies**:
   - If a word has only one subword (e.g., a single letter), its frequency is added directly to `letter_freqs`.
   - If a word has more than one subword, the code:
     - Counts the frequency of the first subword.
     - Iterates through the split representation to create pairs of adjacent subwords and counts their frequencies in `pair_freqs`.
     - At the end of the loop for each word, it also updates the frequency of the last subword.

4. **Compute Pair Scores**:
   - A score is computed for each pair of adjacent letters using the formula:
     \[
     \text{score}(pair) = \frac{\text{pair frequency}}{\text{frequency of the first letter} \times \text{frequency of the second letter}}
     \]
   - This score helps in determining how likely it is that two letters should be merged based on their co-occurrence in the words.

5. **Return Scores**:
   - The scores are returned in a dictionary format where each key is a pair of letters and the value is their computed score.

### Example

Let's apply this code to a small example using the sentences:
- "I love apples"
- "You love bananas"

#### Step 1: Define `word_freqs` and `splits`

First, we assume `word_freqs` and `splits` are as follows:
```python
word_freqs = defaultdict(int, {
    'I': 1,
    'love': 2,
    'apples': 1,
    'bananas': 1
})

splits = {
    'I': ['I'],
    'love': ['l', '##o', '##v', '##e'],
    'apples': ['a', '##p', '##p', '##l', '##e', '##s'],
    'bananas': ['b', '##a', '##n', '##a', '##n', '##a', '##s']
}
```

#### Step 2: Count Frequencies
- For **'I'**:
  - `letter_freqs`: `{'I': 1}`
- For **'love'**:
  - Pairs: ('l', '##o'), ('##o', '##v'), ('##v', '##e')
  - Update:
    - `letter_freqs`: `{'I': 1, 'l': 2, '##o': 2, '##v': 2, '##e': 2}`
    - `pair_freqs`: `{('l', '##o'): 2, ('##o', '##v'): 2, ('##v', '##e'): 2}`
- For **'apples'**:
  - Pairs: ('a', '##p'), ('##p', '##p'), ('##p', '##l'), ('##l', '##e'), ('##e', '##s')
  - Update:
    - `letter_freqs`: `{'I': 1, 'l': 2, '##o': 2, '##v': 2, '##e': 2, 'a': 1, '##p': 3, '##l': 3, '##e': 3, '##s': 1}`
    - `pair_freqs`: `{('l', '##o'): 2, ('##o', '##v'): 2, ('##v', '##e'): 2, ('a', '##p'): 1, ('##p', '##p'): 1, ('##p', '##l'): 1, ('##l', '##e'): 1, ('##e', '##s'): 1}`
- For **'bananas'**:
  - Pairs: ('b', '##a'), ('##a', '##n'), ('##n', '##a'), ('##a', '##n'), ('##n', '##a'), ('##a', '##s')
  - Update:
    - `letter_freqs`: `{'I': 1, 'l': 2, '##o': 2, '##v': 2, '##e': 2, 'a': 2, '##p': 3, '##l': 3, '##e': 3, '##s': 1, 'b': 1, '##n': 2}`
    - `pair_freqs`: `{('l', '##o'): 2, ('##o', '##v'): 2, ('##v', '##e'): 2, ('a', '##p'): 1, ('##p', '##p'): 1, ('##p', '##l'): 1, ('##l', '##e'): 1, ('##e', '##s'): 1, ('b', '##a'): 1, ('##a', '##n'): 1, ('##n', '##a'): 1, ('##a', '##n'): 1, ('##n', '##a'): 1, ('##a', '##s'): 1}`

#### Step 3: Compute Scores

Finally, the scores for merging pairs will be computed as follows:

For example, let's compute the score for the pair `('l', '##o')`:
- Frequency of pair = 2
- Frequency of 'l' = 2
- Frequency of '##o' = 2
- Score for `('l', '##o')`:
\[
\text{score} = \frac{2}{2 \times 2} = 0.5
\]

The final `scores` dictionary will have computed scores for each pair based on the word frequencies.

### Output
After running the code, you will get a dictionary with pairs and their corresponding scores:
```plaintext
Scores for each Pair: {
    ('l', '##o'): 0.5,
    ('##o', '##v'): 0.5,
    ('##v', '##e'): 0.5,
    ('a', '##p'): 0.333,
    ('##p', '##p'): 0.333,
    ('##p', '##l'): 0.25,
    ('##l', '##e'): 0.25,
    ('##e', '##s'): 0.333,
    ('b', '##a'): 1.0,
    ('##a', '##n'): 0.5,
    ('##n', '##a'): 0.5,
    ('##a', '##n'): 0.5,
    ('##n', '##a'): 0.5,
    ('##a', '##s'): 0.5
}
```

### Summary
The `compute_pair_scores` function computes scores for pairs of letters or subwords based on their frequencies within the provided words. This scoring helps to determine which pairs of letters should be merged together in a subsequent step of the tokenization process, allowing for efficient handling of words during the training of models like BERT.

Let's break down the provided code that finds the best scoring pair of subwords or letters to merge and then performs the merging process in a tokenizer context. This process is often part of the Byte Pair Encoding (BPE) algorithm, which is used for creating subword vocabularies for models like BERT.

### Code Explanation

**Code Snippet:**
```python
### finding pair with best score ###
best_pair = ""
max_score = None
for pair, score in pair_scores.items():
    if max_score is None or max_score < score:
        best_pair = pair
        max_score = score

print(best_pair, max_score)

vocab.append("ab")

### merge pair ###
def merge_pair(a, b, splits):
    for word in word_freqs:
        split = splits[word]
        if len(split) == 1:
            continue
        i = 0
        while i < len(split) - 1:
            if split[i] == a and split[i + 1] == b:
                merge = a + b[2:] if b.startswith("##") else a + b
                split = split[:i] + [merge] + split[i + 2 :]
            else:
                i += 1
        splits[word] = split
    return splits

splits = merge_pair("a", "##b", splits)
print(splits["about"])
```

### Breakdown of Steps

#### Step 1: Finding the Best Pair

1. **Initialization**:
   - `best_pair`: A variable to hold the pair of letters/subwords that has the best score found so far.
   - `max_score`: A variable to hold the highest score found for any pair.

2. **Iterate Over Pair Scores**:
   - The code loops through each pair and its corresponding score from the `pair_scores` dictionary.
   - For each `pair`, if `max_score` is `None` or the current `score` is greater than the `max_score`, it updates `best_pair` and `max_score` with the current values.

3. **Print Best Pair**:
   - Finally, it prints the best pair and its corresponding score.

#### Example
Given the sentences **"I love apple banana"** and assuming that the previously computed `pair_scores` have included scores for pairs like `('a', '##b')` with some positive value, the code will find that best scoring pair, e.g., `('a', '##b')` with a score of `0.5`.

#### Step 2: Merging the Best Pair

1. **Function Definition**:
   - The `merge_pair` function takes two parameters, `a` and `b`, which are the letters or subwords to be merged, and `splits`, which is the current representation of words split into subwords.

2. **Iterate Over Words**:
   - The function loops through each word in `word_freqs`. For each word, it retrieves the current `split` representation.

3. **Skip Single Subwords**:
   - If the `split` length is 1 (meaning it's already a single subword), it skips to the next word.

4. **Merging Process**:
   - A while loop is used to iterate through the `split` list. It checks for the occurrence of the pair (`a`, `b`) at adjacent positions.
   - If it finds the pair, it creates a merged version:
     - If `b` starts with `##`, it merges `a` with the part of `b` after `##` (i.e., `b[2:]`).
     - If `b` does not start with `##`, it merges `a` with `b` directly.
   - It updates the `split` list by replacing the pair with the merged version and removing the merged elements.

5. **Update `splits`**:
   - The updated `split` representation is stored back in the `splits` dictionary for that word.

6. **Return Updated Splits**:
   - Finally, the updated `splits` are returned.

#### Example of Merging
Assuming you have:
- The word **"about"** split into `["a", "##b", "##o", "##u", "##t"]`.

When you run `merge_pair("a", "##b", splits)`, it will check for the pair `("a", "##b")`:
- The pair is found at positions `0` and `1`.
- It merges them into `"ab"` (since `"##b"` indicates that `b` is a subword that follows `a`).
- The updated split representation for "about" becomes:
```python
["ab", "##o", "##u", "##t"]
```
#### Final Output
Finally, the code prints the updated split representation of the word "about":
```plaintext
print(splits["about"])  # Output: ['ab', '##o', '##u', '##t']
```

### Summary
1. The first part of the code finds the best pair of letters or subwords to merge based on their frequency scores.
2. The second part defines a function to merge that best pair into the current splits of words. This process continues iteratively, refining the vocabulary and the way words are tokenized for a better representation suitable for training language models like BERT.

Let's break down the provided code snippet step by step. This code is part of the process for building a vocabulary for a tokenizer using a technique similar to Byte Pair Encoding (BPE). The goal is to iteratively merge the most frequent pairs of tokens until the vocabulary reaches a specified size.

### Explanation of the Code

1. **Set Target Vocabulary Size**:
   ```python
   vocab_size = 70
   ```
   This sets the target size of the vocabulary. The loop will continue until the number of unique tokens in the `vocab` reaches this size.

2. **Loop to Merge Token Pairs**:
   ```python
   while len(vocab) < vocab_size:
   ```
   The `while` loop will run as long as the length of the `vocab` is less than `vocab_size`. This means the algorithm will continue to merge pairs of tokens until it has created enough unique tokens in the vocabulary.

3. **Compute Pair Scores**:
   ```python
   scores = compute_pair_scores(splits)
   ```
   The function `compute_pair_scores(splits)` calculates the frequency scores for all adjacent pairs of tokens based on the current splits of words. This step is essential for determining which pairs of tokens are the most common and should be merged next.

4. **Finding the Best Pair**:
   ```python
   best_pair, max_score = "", None
   for pair, score in scores.items():
       if max_score is None or max_score < score:
           best_pair = pair
           max_score = score
   ```
   This loop iterates through the `scores` dictionary to find the pair of tokens with the highest score. The pair with the highest frequency of occurrence (i.e., the most common pair) is identified as `best_pair`.

5. **Merge the Best Pair**:
   ```python
   splits = merge_pair(*best_pair, splits)
   ```
   The `merge_pair` function is called with the tokens in `best_pair`, and it updates the `splits` dictionary to reflect the newly merged token. This process modifies the splits of the words to include the new merged token instead of the two original tokens.

6. **Create and Append New Token**:
   ```python
   new_token = (
       best_pair[0] + best_pair[1][2:]
       if best_pair[1].startswith("##")
       else best_pair[0] + best_pair[1]
   )
   vocab.append(new_token)
   ```
   This code constructs a new token based on the `best_pair`:
   - If the second token in the pair starts with `"##"` (indicating it is a subword), the new token is created by combining the first token with the second token, excluding the `"##"`.
   - If the second token does not start with `"##"`, both tokens are concatenated directly.
   
   The newly created token is then appended to the `vocab`.

7. **Final Output**:
   ```python
   print(f'Final Vocab: {vocab}')
   ```
   After the loop finishes, the final vocabulary is printed. This vocabulary will contain the original tokens along with any new merged tokens created during the process.

### Example of the Process

Let's walk through a simplified example:

1. **Initial Vocabulary**: Suppose you start with a simple vocabulary:
   ```python
   vocab = ["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]", "a", "b", "c"]
   ```

2. **Token Splits**: Let's say you have the following splits:
   ```python
   splits = {
       "apple": ["a", "##p", "##p", "##l", "##e"],
       "banana": ["b", "##a", "##n", "##a", "##n", "##a"]
   }
   ```

3. **Pair Scores Calculation**: The pair scoring might reveal that `("a", "##p")` is the most frequent pair.

4. **Merging**:
   - The best pair `("a", "##p")` is merged into `"ap"`.
   - The updated splits would reflect this merge.

5. **Vocabulary Update**:
   - The new token `"ap"` is appended to the vocabulary.

6. **Repeat**: The loop continues to find the next best pair, calculate scores, merge, and update the vocabulary until it reaches the target size of `70`.

### Summary
The code snippet provides a systematic approach to building a vocabulary through pairwise merging based on frequency, effectively reducing the number of tokens while capturing the most common combinations in the dataset. This method is useful in natural language processing tasks to create efficient tokenizers.

Let's break down the provided code step by step to understand what each part does in the context of training a custom BERT tokenizer using the `BertWordPieceTokenizer` class from the Hugging Face library.

### Code Explanation

1. **Initialize the Tokenizer**:
   ```python
   tokenizer = BertWordPieceTokenizer(
       clean_text=True,
       handle_chinese_chars=False,
       strip_accents=False,
       lowercase=True
   )
   ```
   - **`BertWordPieceTokenizer`**: This initializes a WordPiece tokenizer specifically for BERT.
   - **Parameters**:
     - **`clean_text=True`**: Cleans the input text by removing unwanted characters or formatting.
     - **`handle_chinese_chars=False`**: Determines whether to handle Chinese characters explicitly. Set to `False` means it won't treat them differently.
     - **`strip_accents=False`**: When set to `False`, accents are preserved in the text (e.g., é remains as é).
     - **`lowercase=True`**: Converts all input text to lowercase, which is common for uncased models.

2. **Training the Tokenizer**:
   ```python
   tokenizer.train(
       files=paths,
       vocab_size=30_000,
       min_frequency=5,
       limit_alphabet=1000,
       wordpieces_prefix='##',
       special_tokens=['[PAD]', '[CLS]', '[SEP]', '[MASK]', '[UNK]']
   )
   ```
   - **`tokenizer.train(...)`**: This method trains the tokenizer on the specified datasets.
   - **Parameters**:
     - **`files=paths`**: Specifies the files containing the text data for training.
     - **`vocab_size=30_000`**: Sets the maximum vocabulary size to 30,000 tokens.
     - **`min_frequency=5`**: A token must appear at least 5 times in the training data to be included in the vocabulary.
     - **`limit_alphabet=1000`**: Limits the number of unique characters (the alphabet) to 1000.
     - **`wordpieces_prefix='##'`**: Defines the prefix for subword tokens; typically, in WordPiece, subwords are prefixed with "##".
     - **`special_tokens=['[PAD]', '[CLS]', '[SEP]', '[MASK]', '[UNK]']`**: Specifies special tokens used in the tokenizer.

3. **Create a Directory for the Model**:
   ```python
   os.mkdir('./bert-it-1')
   ```
   This line creates a new directory named `bert-it-1` to store the tokenizer model files.

4. **Save the Trained Model**:
   ```python
   tokenizer.save_model('./bert-it-1', 'bert-it')
   ```
   - This saves the trained tokenizer model in the `bert-it-1` directory with the prefix `bert-it`. The files saved include the vocabulary and configuration needed to load the tokenizer later.

5. **Load the Trained Tokenizer**:
   ```python
   tokenizer = BertTokenizer.from_pretrained('./bert-it-1/bert-it-vocab.txt', local_files_only=True)
   ```
   - This line loads the saved tokenizer from the specified path. It uses the vocabulary file that was generated during the training process.

6. **Tokenize a Sample Sentence**:
   ```python
   token_ids = tokenizer('I like surfboarding!')['input_ids']
   ```
   - This tokenizes the input sentence "I like surfboarding!" and returns the corresponding token IDs (numeric representation of the tokens).

7. **Print Token IDs and Converted Tokens**:
   ```python
   print(token_ids)
   print(tokenizer.convert_ids_to_tokens(token_ids))
   ```
   - The first `print` statement outputs the numeric IDs of the tokens generated from the input sentence.
   - The second `print` statement converts those token IDs back into their string representations (i.e., the original tokens) and prints them.

### Summary
The provided code trains a custom WordPiece tokenizer for BERT using a specified dataset, creates a vocabulary, saves the tokenizer model, and demonstrates how to tokenize a sample sentence. This is useful for NLP tasks where a model needs to process text data consistently with its training and tokenization methods.

By training your tokenizer, you can adapt it to specific language use cases, dialects, or domains that may not be adequately covered by pre-trained tokenizers.